In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as md
import numpy as np
from datetime import date
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import svm, datasets
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
df = pd.read_csv('/content/gdrive/My Drive/Research/AWS/modify_date_price2.csv')
df.columns = [column.replace(' ', '_') for column in df.columns]
df = df.loc[df.Instance_Family == "General purpose"]
df.head()

Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,6,7,8,9,11,12,13,14,15,16,17,18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed:_0,EffectiveDates,PricePerUnit,LeaseContractLength,PurchaseOption,OfferingClass,Location,Instance_Type,Current_Generation,Instance_Family,vCPU,Clock_Speed,Memory,Storage,Network_Performance,Operating_System,Tenancy,usageType,Dedicated_EBS_Throughput,Normalization_Size_Factor,Processor_Features
0,0.0,4/30/2017,0.01,1yr,No Upfront,standard,US East (Ohio),m4.xlarge,Yes,General purpose,4.0,2.4 GHz,16 GiB,EBS only,High,RHEL,Dedicated,USE2-DedicatedUsage:m4.xlarge,750 Mbps,8.0,Intel AVX; Intel AVX2; Intel Turbo
1,1.0,3/31/2018,0.01,3yr,Partial Upfront,convertible,US East (N. Virginia),m4.large,Yes,General purpose,2.0,2.4 GHz,8 GiB,EBS only,Moderate,Linux,Dedicated,DedicatedUsage:m4.large,450 Mbps,4.0,Intel AVX; Intel AVX2; Intel Turbo
4,4.0,7/1/2019,0.01,OnDemand,OnDemand,OnDemand,US West (N. California),m3.xlarge,No,General purpose,4.0,2.5 GHz,15 GiB,2 x 40 SSD,High,RHEL,Shared,USW1-BoxUsage:m3.xlarge,NaN,8.0,Intel AVX; Intel Turbo
5,5.0,3/1/2019,0.01,3yr,All Upfront,convertible,US East (N. Virginia),m5ad.2xlarge,Yes,General purpose,8.0,2.5 GHz,32 GiB,1 x 300 NVMe SSD,Up to 10 Gigabit,Windows,Shared,BoxUsage:m5ad.2xlarge,Upto 2120 Mbps,16.0,"AVX, AVX2, AMD Turbo"
8,8.0,10/1/2018,0.01,1 yr,AllUpfront,standard,US West (Oregon),a1,Yes,General purpose,16.0,2.3 GHz,NaN,NaN,NaN,Linux,Reserved,USW2-ReservedHostUsage:a1,NaN,NaN,NaN


In [3]:
# df["Memory"] = df["Memory"].astype("float64")
df["EffectiveDates"] = df["EffectiveDates"].astype("datetime64")
# df["Project_Type"] = df["Project_Type"].astype(str)
# df["Storage"] = df["Storage"].astype(str)
df["Processor_Archi"] = df["Storage"].astype(str)
# df["Clock_Speed"] = df["Clock_Speed"].astype(str)
df["PricePerUnit"] = df["PricePerUnit"].astype(float)
df['Year'] = pd.DatetimeIndex(df['EffectiveDates']).year.astype("int32")
df['Month'] = pd.DatetimeIndex(df['EffectiveDates']).month
# columns = ['PricePerUnit', 'UpfrontCost', 'Instance_Type', 'Operating_System', 'Location', 'LeaseContractLength', 'PurchaseOption', 'OfferingClass', 'Year', 'Month']
columns = ['PricePerUnit', 'UpfrontCost', 'Instance_Type', 'Operating_System', 'Location', 'LeaseContractLength', 'PurchaseOption', 'OfferingClass']
df1 = df.reindex(columns=columns)
df1 = df1[columns]
df1.head()

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass
0,0.01,NaN,m4.xlarge,RHEL,US East (Ohio),1yr,No Upfront,standard
1,0.01,NaN,m4.large,Linux,US East (N. Virginia),3yr,Partial Upfront,convertible
4,0.01,NaN,m3.xlarge,RHEL,US West (N. California),OnDemand,OnDemand,OnDemand
5,0.01,NaN,m5ad.2xlarge,Windows,US East (N. Virginia),3yr,All Upfront,convertible
8,0.01,NaN,a1,Linux,US West (Oregon),1 yr,AllUpfront,standard


In [4]:
# series = df1.isnull().sum().sort_values(ascending=False)
# for index, value in series.items() :
#     if value > (df1.shape[0] > 0) :
#         print(str(index) + " : " + str(value))

In [5]:
df1['Operating_System'] = df1['Operating_System'].replace({np.nan: 'Windows'})
df1.isnull().sum()

PricePerUnit                0
UpfrontCost            253655
Instance_Type               0
Operating_System            0
Location                    0
LeaseContractLength         0
PurchaseOption              0
OfferingClass               0
dtype: int64

In [6]:
def setOnDemandPrice (row):
    if (row['Instance_Type'] == 'a1.medium') :
      return 0.0255
    if (row['Instance_Type'] == 'a1.large') :
      return 0.051
    if (row['Instance_Type'] == 'a1.xlarge') :
      return 0.102
    if (row['Instance_Type'] == 'a1.2xlarge') :
      return 0.204  
    if (row['Instance_Type'] == 'a1.4xlarge') :
      return 0.408
    if (row['Instance_Type'] == 'a1.metal') :
      return 0.408  
    if (row['Instance_Type'] == 't4g.nano') :
      return 0.0042  
    if (row['Instance_Type'] == 't4g.micro') :
      return 0.0084         
    if (row['Instance_Type'] == 't4g.small') :
      return 0.0168
    if (row['Instance_Type'] == 't4g.medium') :
      return 0.0336        
    if (row['Instance_Type'] == 't4g.large') :
      return 0.0672 
    if (row['Instance_Type'] == 't4g.xlarge') :
      return 0.1344       
    if (row['Instance_Type'] == 't4g.2xlarge') :
      return 0.2688  

    if (row['Instance_Type'] == 't3.nano') :
      return 0.0052  
    if (row['Instance_Type'] == 't3.micro') :
      return 0.0104         
    if (row['Instance_Type'] == 't3.small') :
      return 0.0208
    if (row['Instance_Type'] == 't3.medium') :
      return 0.0416        
    if (row['Instance_Type'] == 't3.large') :
      return 0.0832 
    if (row['Instance_Type'] == 't3.xlarge') :
      return 0.1664       
    if (row['Instance_Type'] == 't3.2xlarge') :
      return 0.3328 

    if (row['Instance_Type'] == 't3a.nano') :
      return 0.0047  
    if (row['Instance_Type'] == 't3a.micro') :
      return 0.0094         
    if (row['Instance_Type'] == 't3a.small') :
      return 0.0188
    if (row['Instance_Type'] == 't3a.medium') :
      return 0.0376        
    if (row['Instance_Type'] == 't3a.large') :
      return 0.0752 
    if (row['Instance_Type'] == 't3a.xlarge') :
      return 0.1504       
    if (row['Instance_Type'] == 't3a.2xlarge') :
      return 0.3008  

    if (row['Instance_Type'] == 'm6g.medium') :
      return 0.0385        
    if (row['Instance_Type'] == 'm6g.large') :
      return 0.077 
    if (row['Instance_Type'] == 'm6g.xlarge') :
      return 0.154       
    if (row['Instance_Type'] == 'm6g.2xlarge') :
      return 0.308
    if (row['Instance_Type'] == 'm6g.4xlarge') :
      return 0.616
    if (row['Instance_Type'] == 'm6g.8xlarge') :
      return 1.232
    if (row['Instance_Type'] == 'm6g.12xlarge') :
      return 1.848        
    if (row['Instance_Type'] == 'm6g.16xlarge') :
      return 2.464    
    if (row['Instance_Type'] == 'm6g.metal') :
      return 2.464    

    if (row['Instance_Type'] == 'm6gd.medium') :
      return 0.0452       
    if (row['Instance_Type'] == 'm6gd.large') :
      return 0.0904
    if (row['Instance_Type'] == 'm6gd.xlarge') :
      return 0.1808     
    if (row['Instance_Type'] == 'm6gd.2xlarge') :
      return 0.3616
    if (row['Instance_Type'] == 'm6gd.4xlarge') :
      return 0.7232
    if (row['Instance_Type'] == 'm6gd.8xlarge') :
      return 1.4464
    if (row['Instance_Type'] == 'm6gd.12xlarge') :
      return 2.1696       
    if (row['Instance_Type'] == 'm6gd.16xlarge') :
      return 2.8928   
    if (row['Instance_Type'] == 'm6gd.metal') :
      return 2.8928  

    if (row['Instance_Type'] == 'm5.large') :
      return 0.0906
    if (row['Instance_Type'] == 'm5.xlarge') :
      return 0.192    
    if (row['Instance_Type'] == 'm5.2xlarge') :
      return 0.384
    if (row['Instance_Type'] == 'm5.4xlarge') :
      return 0.768
    if (row['Instance_Type'] == 'm5.8xlarge') :
      return 1.536
    if (row['Instance_Type'] == 'm5.12xlarge') :
      return 2.304       
    if (row['Instance_Type'] == 'm5.16xlarge') :
      return 3.072  
    if (row['Instance_Type'] == 'm5.24xlarge') :
      return 4.608    
    if (row['Instance_Type'] == 'm5.metal') :
      return 4.608    

    if (row['Instance_Type'] == 'm5d.large') :
      return 0.113
    if (row['Instance_Type'] == 'm5d.xlarge') :
      return 0.226    
    if (row['Instance_Type'] == 'm5d.2xlarge') :
      return 0.452
    if (row['Instance_Type'] == 'm5d.4xlarge') :
      return 0.904
    if (row['Instance_Type'] == 'm5d.8xlarge') :
      return 1.808
    if (row['Instance_Type'] == 'm5d.12xlarge') :
      return 2.712       
    if (row['Instance_Type'] == 'm5d.16xlarge') :
      return 3.616 
    if (row['Instance_Type'] == 'm5d.24xlarge') :
      return 5.424    
    if (row['Instance_Type'] == 'm5d.metal') :
      return 5.424  

    if (row['Instance_Type'] == 'm5a.large') :
      return 0.086
    if (row['Instance_Type'] == 'm5a.xlarge') :
      return 0.172    
    if (row['Instance_Type'] == 'm5a.2xlarge') :
      return 0.688
    if (row['Instance_Type'] == 'm5a.4xlarge') :
      return 0.688
    if (row['Instance_Type'] == 'm5a.8xlarge') :
      return 1.376

    if (row['Instance_Type'] == 'm5ad.large') :
      return 0.103
    if (row['Instance_Type'] == 'm5ad.xlarge') :
      return 0.206    
    if (row['Instance_Type'] == 'm5ad.2xlarge') :
      return 0.412
    if (row['Instance_Type'] == 'm5ad.4xlarge') :
      return 0.824
    if (row['Instance_Type'] == 'm5ad.12xlarge') :
      return 2.472      
    if (row['Instance_Type'] == 'm5ad.24xlarge') :
      return 4.944    

    if (row['Instance_Type'] == 'm5n.large') :
      return 0.119
    if (row['Instance_Type'] == 'm5n.xlarge') :
      return 0.238    
    if (row['Instance_Type'] == 'm5n.2xlarge') :
      return 0.476
    if (row['Instance_Type'] == 'm5n.4xlarge') :
      return 0.952
    if (row['Instance_Type'] == 'm5n.8xlarge') :
      return 1.904
    if (row['Instance_Type'] == 'm5n.12xlarge') :
      return 2.856       
    if (row['Instance_Type'] == 'm5n.16xlarge') :
      return 3.808
    if (row['Instance_Type'] == 'm5n.24xlarge') :
      return 5.712

    if (row['Instance_Type'] == 'm4.large') :
      return 0.1
    if (row['Instance_Type'] == 'm4.xlarge') :
      return 0.2   
    if (row['Instance_Type'] == 'm4.2xlarge') :
      return 0.4
    if (row['Instance_Type'] == 'm4.4xlarge') :
      return 0.8
    if (row['Instance_Type'] == 'm4.10xlarge') :
      return 2   
    if (row['Instance_Type'] == 'm4.16xlarge') :
      return 3.2 
    else :
      return 0.3008    
                          

def get_a1_ec2(row):
  if (row['Instance_Type'] == 'a1.medium' or row['Instance_Type'] == 'a1') :
        return 'a1.medium' 
  if (row['Instance_Type'] == 'a1.large' or row['Instance_Type'] == 'a1.xlarge' or row['Instance_Type'] == 'a1.2xlarge' or row['Instance_Type'] == 'a1.4xlarge' or row['Instance_Type'] == 'a1.metal') :
        return row['Instance_Type']   

def get_t4g_ec2(row):     
    if (row['Instance_Type'] == 't4g.nano') :
        return row['Instance_Type']
    if (row['Instance_Type'] == 't4g.micro') :
        return row['Instance_Type']        
    if (row['Instance_Type'] == 't4g.small') :
        return row['Instance_Type']
    if (row['Instance_Type'] == 't4g.medium') :
        return row['Instance_Type']       
    if (row['Instance_Type'] == 't4g.large') :
        return row['Instance_Type']
    if (row['Instance_Type'] == 't4g.xlarge') :
        return row['Instance_Type']     
    if (row['Instance_Type'] == 't4g.2xlarge') :
        return row['Instance_Type']

def get_t3_ec2(row):  
    if (row['Instance_Type'] == 't3.nano') :
        return row['Instance_Type']
    if (row['Instance_Type'] == 't3.micro') :
        return row['Instance_Type']        
    if (row['Instance_Type'] == 't3.small') :
        return row['Instance_Type']
    if (row['Instance_Type'] == 't3.medium') :
        return row['Instance_Type']        
    if (row['Instance_Type'] == 't3.large') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 't3.xlarge') :
      return row['Instance_Type']    
    if (row['Instance_Type'] == 't3.2xlarge') :
      return row['Instance_Type']   


def get_t3a_ec2(row):      
    if (row['Instance_Type'] == 't3a.nano') :
      return row['Instance_Type'] 
    if (row['Instance_Type'] == 't3a.micro') :
      return row['Instance_Type']         
    if (row['Instance_Type'] == 't3a.small') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 't3a.medium') :
      return row['Instance_Type']       
    if (row['Instance_Type'] == 't3a.large') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 't3a.xlarge') :
      return row['Instance_Type']     
    if (row['Instance_Type'] == 't3a.2xlarge') :
      return row['Instance_Type']

def get_m6g_ec2(row):      
    if (row['Instance_Type'] == 'm6g.medium') :
      return row['Instance_Type']        
    if (row['Instance_Type'] == 'm6g.large') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm6g.xlarge') :
      return row['Instance_Type']    
    if (row['Instance_Type'] == 'm6g.2xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm6g.4xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm6g.8xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm6g.12xlarge') :
      return row['Instance_Type']       
    if (row['Instance_Type'] == 'm6g.16xlarge') :
      return row['Instance_Type']  
    if (row['Instance_Type'] == 'm6g.metal') :
      return row['Instance_Type'] 

def get_m5_ec2(row):      
    if (row['Instance_Type'] == 'm5.large') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5.xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5.2xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5.4xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5.8xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5.12xlarge') :
      return row['Instance_Type']      
    if (row['Instance_Type'] == 'm5.16xlarge') :
      return row['Instance_Type']  
    if (row['Instance_Type'] == 'm5.24xlarge') :
      return row['Instance_Type']   
    if (row['Instance_Type'] == 'm5.metal') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5d.large') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5d.xlarge') :
      return row['Instance_Type']    
    if (row['Instance_Type'] == 'm5d.2xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5d.4xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5d.8xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5d.12xlarge') :
      return row['Instance_Type']      
    if (row['Instance_Type'] == 'm5d.16xlarge') :
      return row['Instance_Type'] 
    if (row['Instance_Type'] == 'm5d.24xlarge') :
      return row['Instance_Type']    
    if (row['Instance_Type'] == 'm5d.metal') :
      return row['Instance_Type']   


def get_m5a_ec2(row):  
    if (row['Instance_Type'] == 'm5a.large') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5a.xlarge') :
      return row['Instance_Type']  
    if (row['Instance_Type'] == 'm5a.2xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5a.4xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5a.8xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5ad.large') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5ad.xlarge') :
      return row['Instance_Type']    
    if (row['Instance_Type'] == 'm5ad.2xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5ad.4xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5ad.12xlarge') :
      return row['Instance_Type']     
    if (row['Instance_Type'] == 'm5ad.24xlarge') :
      return row['Instance_Type']


def get_m5n_ec2(row):  
    if (row['Instance_Type'] == 'm5n.large') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5n.xlarge') :
      return row['Instance_Type']   
    if (row['Instance_Type'] == 'm5n.2xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5n.4xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5n.8xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5n.12xlarge') :
      return row['Instance_Type']      
    if (row['Instance_Type'] == 'm5n.16xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm5n.24xlarge') :
      return row['Instance_Type']


def get_m4_ec2(row): 
    if (row['Instance_Type'] == 'm4.large') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm4.xlarge') :
      return row['Instance_Type']  
    if (row['Instance_Type'] == 'm4.2xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm4.4xlarge') :
      return row['Instance_Type']
    if (row['Instance_Type'] == 'm4.10xlarge') :
      return row['Instance_Type']   
    if (row['Instance_Type'] == 'm4.16xlarge') :
      return row['Instance_Type'] 


def setInstanceType (row):
    if (row['Instance_Type'] == 'a1.medium') :
        return 10
    if (row['Instance_Type'] == 'a1.large') :
        return 15
    if (row['Instance_Type'] == 'a1.xlarge') :
        return 24
    if (row['Instance_Type'] == 'a1.2xlarge') :
        return 37  
    if (row['Instance_Type'] == 'a1.4xlarge') :
        return 46
    if (row['Instance_Type'] == 'a1.metal') :
        return 47  
    if (row['Instance_Type'] == 't4g.nano') :
        return 1 
    if (row['Instance_Type'] == 't4g.micro') :
        return 4        
    if (row['Instance_Type'] == 't4g.small') :
        return 7
    if (row['Instance_Type'] == 't4g.medium') :
        return 11       
    if (row['Instance_Type'] == 't4g.large') :
        return 16
    if (row['Instance_Type'] == 't4g.xlarge') :
        return 31     
    if (row['Instance_Type'] == 't4g.2xlarge') :
        return 42

    if (row['Instance_Type'] == 't3.nano') :
        return 3 
    if (row['Instance_Type'] == 't3.micro') :
        return 6         
    if (row['Instance_Type'] == 't3.small') :
        return 9
    if (row['Instance_Type'] == 't3.medium') :
        return 13        
    if (row['Instance_Type'] == 't3.large') :
      return 19
    if (row['Instance_Type'] == 't3.xlarge') :
      return 30    
    if (row['Instance_Type'] == 't3.2xlarge') :
      return 44 

    if (row['Instance_Type'] == 't3a.nano') :
      return 36 
    if (row['Instance_Type'] == 't3a.micro') :
      return 5         
    if (row['Instance_Type'] == 't3a.small') :
      return 8
    if (row['Instance_Type'] == 't3a.medium') :
      return 12       
    if (row['Instance_Type'] == 't3a.large') :
      return 17
    if (row['Instance_Type'] == 't3a.xlarge') :
      return 32     
    if (row['Instance_Type'] == 't3a.2xlarge') :
      return 43  

    if (row['Instance_Type'] == 'm6g.medium') :
      return 14        
    if (row['Instance_Type'] == 'm6g.large') :
      return 18
    if (row['Instance_Type'] == 'm6g.xlarge') :
      return 29     
    if (row['Instance_Type'] == 'm6g.2xlarge') :
      return 46
    if (row['Instance_Type'] == 'm6g.4xlarge') :
      return 50
    if (row['Instance_Type'] == 'm6g.8xlarge') :
      return 60
    if (row['Instance_Type'] == 'm6g.12xlarge') :
      return 63       
    if (row['Instance_Type'] == 'm6g.16xlarge') :
      return 69   
    if (row['Instance_Type'] == 'm6g.metal') :
      return 70 

    if (row['Instance_Type'] == 'm6gd.medium') :
      return 15     
    if (row['Instance_Type'] == 'm6gd.large') :
      return 22
    if (row['Instance_Type'] == 'm6gd.xlarge') :
      return 33   
    if (row['Instance_Type'] == 'm6gd.2xlarge') :
      return 49
    if (row['Instance_Type'] == 'm6gd.4xlarge') :
      return 55
    if (row['Instance_Type'] == 'm6gd.8xlarge') :
      return 62
    if (row['Instance_Type'] == 'm6gd.12xlarge') :
      return 67     
    if (row['Instance_Type'] == 'm6gd.16xlarge') :
      return 73   
    if (row['Instance_Type'] == 'm6gd.metal') :
      return 74

    if (row['Instance_Type'] == 'm5.large') :
      return 21
    if (row['Instance_Type'] == 'm5.xlarge') :
      return 28
    if (row['Instance_Type'] == 'm5.2xlarge') :
      return 47
    if (row['Instance_Type'] == 'm5.4xlarge') :
      return 54
    if (row['Instance_Type'] == 'm5.8xlarge') :
      return 64
    if (row['Instance_Type'] == 'm5.12xlarge') :
      return 68      
    if (row['Instance_Type'] == 'm5.16xlarge') :
      return 76  
    if (row['Instance_Type'] == 'm5.24xlarge') :
      return 80   
    if (row['Instance_Type'] == 'm5.metal') :
      return 81   

    if (row['Instance_Type'] == 'm5d.large') :
      return 25
    if (row['Instance_Type'] == 'm5d.xlarge') :
      return 38    
    if (row['Instance_Type'] == 'm5d.2xlarge') :
      return 48
    if (row['Instance_Type'] == 'm5d.4xlarge') :
      return 56
    if (row['Instance_Type'] == 'm5d.8xlarge') :
      return 64
    if (row['Instance_Type'] == 'm5d.12xlarge') :
      return 72      
    if (row['Instance_Type'] == 'm5d.16xlarge') :
      return 78 
    if (row['Instance_Type'] == 'm5d.24xlarge') :
      return 83    
    if (row['Instance_Type'] == 'm5d.metal') :
      return 84 

    if (row['Instance_Type'] == 'm5a.large') :
      return 22
    if (row['Instance_Type'] == 'm5a.xlarge') :
      return 34  
    if (row['Instance_Type'] == 'm5a.2xlarge') :
      return 52
    if (row['Instance_Type'] == 'm5a.4xlarge') :
      return 53
    if (row['Instance_Type'] == 'm5a.8xlarge') :
      return 61

    if (row['Instance_Type'] == 'm5ad.large') :
      return 26
    if (row['Instance_Type'] == 'm5ad.xlarge') :
      return 39    
    if (row['Instance_Type'] == 'm5ad.2xlarge') :
      return 47
    if (row['Instance_Type'] == 'm5ad.4xlarge') :
      return 58
    if (row['Instance_Type'] == 'm5ad.12xlarge') :
      return 71     
    if (row['Instance_Type'] == 'm5ad.24xlarge') :
      return 82   

    if (row['Instance_Type'] == 'm5n.large') :
      return 27
    if (row['Instance_Type'] == 'm5n.xlarge') :
      return 41   
    if (row['Instance_Type'] == 'm5n.2xlarge') :
      return 51
    if (row['Instance_Type'] == 'm5n.4xlarge') :
      return 59
    if (row['Instance_Type'] == 'm5n.8xlarge') :
      return 65
    if (row['Instance_Type'] == 'm5n.12xlarge') :
      return 75      
    if (row['Instance_Type'] == 'm5n.16xlarge') :
      return 79
    if (row['Instance_Type'] == 'm5n.24xlarge') :
      return 85

    if (row['Instance_Type'] == 'm4.large') :
      return 23
    if (row['Instance_Type'] == 'm4.xlarge') :
      return 35  
    if (row['Instance_Type'] == 'm4.2xlarge') :
      return 45
    if (row['Instance_Type'] == 'm4.4xlarge') :
      return 55
    if (row['Instance_Type'] == 'm4.10xlarge') :
      return 66   
    if (row['Instance_Type'] == 'm4.16xlarge') :
      return 77 
    else :
      return 43

os_types = {
                 'Windows':2,
                 'Linux':1,
                 'SUSE':3,
                 'RHEL':4
                 }  
                       

location_type = {
      'Asia Pacific (Mumbai)':1,    
      'US East (Ohio)':2,     
      'US East (N. Virginia)':3, 
      'US West (Oregon)':4,
      'EU (Ireland)':5,
      'EU (Frankfurt)':6,
      'Canada (Central)':7,
      'EU (London)':9,
      'EU (Paris)':8,
      'EU (Stockholm)':10,
      'Asia Pacific (Tokyo)':11,   
      'AWS GovCloud (US-East)':12,
      'AWS GovCloud (US)':13,
      'US West (N. California)':14,
      'Asia Pacific (Seoul)':15,  
      'Asia Pacific (Singapore)':16,
      'Asia Pacific (Sydney)':17,
      'EU (Milan)':18,
      'Middle East (Bahrain)':20,
      'Asia Pacific (Cape Town)':19,
      'Asia Pacific (Osaka-Local)':21,
      'Asia Pacific (Hong Kong)':22,
      'South America (Sao Paulo)':23          
}     
  
purchase_type = {
    'NoUpfront':'NoUpfront',            
    'PartialUpfront':'PartialUpfront',        
    'AllUpfront':'AllUpfront',
    'No Upfront':'NoUpfront',            
    'Partial Upfront':'PartialUpfront',        
    'All pfront':'AllUpfront',
    'OnDemand': 'OnDemand'          
}

offeringClass = {
    'standard':1,
    'convertible':2,
    'OnDemand': 3
}

leaseContractLength = {
    '1yr':'1yr',
    '3yr':'3yr',
    '1 yr':'1yr',
    '3 yr':'3yr',
    'OnDemand': 'OnDemand'
}

clockSpeed = {
    'Up to 3.3 GHz':1,
    'Up to 3.0 GHz':2,      
    '2.3 GHz':3,  
    '2.4 GHz': 4,               
    '2.5 GHz':5         
}

year = {
    2016:1,
    2017:2,      
    2018:3,  
    2019:4,               
    2020:5         
}


def setOS (row):
       if (row['Operating_System'] == None) :
          return 'Windows'
       else :
          return row['Operating_System']       

def setLocation (row):
       if (row['Location'] == None) :
          return 'US West (Oregon)'
       else :
          return row['Location']                        


def setPurchaseOption (row):
       if (row['PurchaseOption'] == None) :
          return 'AllUpfront'
       if (row['PurchaseOption'] == 'No Upfront') :
          return 'NoUpfront'
       if (row['PurchaseOption'] == 'Partial Upfront') :
          return 'PartialUpfront'
       if (row['PurchaseOption'] == 'All Upfront') :
          return 'AllUpfront'      
       else :
          return row['PurchaseOption']  

def setOfferingClass (row):
       if (row['OfferingClass'] == None) :
          return 'standard'
       else :
          return row['OfferingClass']       

def setLeaseContractLength (row):
       if (row['LeaseContractLength'] == None) :
          return 3
       else :
          return row['LeaseContractLength']    

# df1['Storage'] = df1.apply (lambda row: setStorage(row), axis=1)   
# df1['Storage'] = df1.apply (lambda row: setStorageSize(row), axis=1)  

# df1['Instance_Type'] = df1.apply (lambda row: groupedInstance_Type(row), axis=1)
# df1['Instance_Type'] = df1.Instance_Type.map (instant_types)

df1['Operating_System'] = df1.apply (lambda row: setOS(row), axis=1)
# df1['Operating_System'] = df1.Operating_System.map (os_types)

df1['Location'] = df1.apply (lambda row: setLocation(row), axis=1)
# df1['Location'] = df1.Location.map (location_type)
df1['Location'] = df1['Location'].fillna('US West (Oregon)')

# df1['Instance_Type'] = df1.apply (lambda row: setInstanceType(row), axis=1)

df1['PurchaseOption'] = df1.apply (lambda row: setPurchaseOption(row), axis=1)
df1['PurchaseOption'] = df1.PurchaseOption.map (purchase_type)

df1['OfferingClass'] = df1.apply (lambda row: setOfferingClass(row), axis=1)
# df1['OfferingClass'] = df1.OfferingClass.map (offeringClass)

df1['LeaseContractLength'] = df1.apply (lambda row: setLeaseContractLength(row), axis=1)
df1['LeaseContractLength'] = df1.LeaseContractLength.map (leaseContractLength)

# df1['Year'] = df1.Year.map (year)
# df1['Location'] = df1['Location'].fillna(4).astype(np.int64)

df1.head(5) 

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass
0,0.01,NaN,m4.xlarge,RHEL,US East (Ohio),1yr,NoUpfront,standard
1,0.01,NaN,m4.large,Linux,US East (N. Virginia),3yr,PartialUpfront,convertible
4,0.01,NaN,m3.xlarge,RHEL,US West (N. California),OnDemand,OnDemand,OnDemand
5,0.01,NaN,m5ad.2xlarge,Windows,US East (N. Virginia),3yr,AllUpfront,convertible
8,0.01,NaN,a1,Linux,US West (Oregon),1yr,AllUpfront,standard


In [7]:
def setOSPrice(x):
       if (x == 'Linux') :
          return 100
       if (x == 'Windows') :
          return 114.64 
       if (x == 'SUSE') :
          return 193
       if (x == 'RHEL') :
          return 166.67        
       else :
          return 100 


def setOfferingClassPrice(OfferingClass, price) :
  if(OfferingClass == 'convertible'):
    return price*114.77/100
  else:
    return price

def setStoragePriceWithLocation(x):
  if(x == 'Asia Pacific (Mumbai)' or x == 'Asia Pacific (Seoul)'):
    return 114
  if(x == 'US East (Ohio)'):
    return 100
  if(x == 'US East (N. Virginia)' or x == 'US West (Oregon)'):
    return 80
  if(x == 'EU (Ireland)'):
    return 88  
  if(x == 'EU (Frankfurt)' or x == 'Asia Pacific (Sydney)'):
    return 119
  if(x == 'Canada (Central)'):
    return 110
  if(x == 'EU (Paris)'):
    return 107
  if(x == 'EU (London)'):
    return 92.8
  if(x == 'EU (Stockholm)'):
    return 104.5
  if(x == 'Asia Pacific (Tokyo)' or x == 'US West (N. California)' or x == 'Asia Pacific (Singapore)'):
    return 120 
  if(x == 'AWS GovCloud (US-East)' or x == 'AWS GovCloud (US)' or x == 'Asia Pacific (Osaka-Local)'):
    return 96
  if(x == 'EU (Milan)'):
    return 92.4
  if(x == 'Asia Pacific (Cape Town)'):
    return 126.4
  if(x == 'Middle East (Bahrain)'):
    return 130.9
  if(x == 'Asia Pacific (Hong Kong)'):
    return 132
  if(x == 'South America (Sao Paulo)'):
    return 152
  else:  
    return 100   
  

def setLocationPrice(x):
  if(x == 'Asia Pacific (Mumbai)'):
    return 66.67
  if(x == 'US East (Ohio)' or x == 'US East (N. Virginia)' or x == 'US West (Oregon)'):
    return 100
  if(x == 'EU (Ireland)'):
    return 109.52
  if(x == 'EU (Frankfurt)'):
    return 114.29  
  if(x == 'Canada (Central)' or x == 'EU (Paris)'):
    return 124.26
  if(x == 'EU (London)'):
    return 126.41
  if(x == 'EU (Stockholm)' or x == 'Asia Pacific (Tokyo)'):
    return 128.57
  if(x == 'AWS GovCloud (US-East)' or x == 'AWS GovCloud (US)'):
    return 130.73
  if(x == 'US West (N. California)'):
    return 132.89
  if(x == 'Asia Pacific (Seoul)'):
    return 139.29 
  if(x == 'Asia Pacific (Singapore)'):
    return 140.47
  if(x == 'Asia Pacific (Sydney)'):
    return 141.44
  if(x == 'EU (Milan)' or x == 'Asia Pacific (Cape Town)'):
    return 142.63
  if(x == 'Middle East (Bahrain)'):
    return 149.26
  if(x == 'Asia Pacific (Osaka-Local)'):
    return 161.90
  if(x == 'Asia Pacific (Hong Kong)'):
    return 173.81
  if(x == 'South America (Sao Paulo)'):
    return 179.99
  else:  
    return 100  

def getHourlyCostWithPurchaseType(purchase_type,leaseContractLength, onDemandPrice):
  NumHours = 365*24*1   #365 days * 24 hours * 1 year = 8760.0000 hours
  if(leaseContractLength == '3yr'):
    NumHours = 365*24*3   #365 days * 24 hours * 1 year = 8760.0000 hours

  y = onDemandPrice
  if(leaseContractLength == '1yr'):
    y = onDemandPrice*62.73/100
  if(leaseContractLength == '3yr'):
    y = onDemandPrice*43.15/100
  TotalCommitment = y * NumHours
  x = 0
  if(purchase_type == 'PartialUpfront'):
   x = 0.5
  if(purchase_type == 'NoUpfront'):
   x = 1 
  UpfrontCost = x*TotalCommitment
  HourlyCost = (TotalCommitment - UpfrontCost)/NumHours
  return HourlyCost

def getUpfrontCostWithPurchaseType(purchase_type,leaseContractLength, onDemandPrice):
  if(leaseContractLength == '3yr'):
    NumHours = 365*24*3 

  NumHours = 365*24*1    

  y = onDemandPrice
  if(leaseContractLength == '1yr'):
    y = onDemandPrice*62.73/100
  if(leaseContractLength == '3yr'):
    y = onDemandPrice*43.15/100

  TotalCommitment = y * NumHours
  x = 0
  if(purchase_type == 'PartialUpfront'):
   x = 0.5
  if(purchase_type == 'NoUpfront'):
   x = 1 
  UpfrontCost = x*TotalCommitment
  return UpfrontCost

def setPricingEC2(row):
  # if (row['PricePerUnit'] <= 5 or row['PricePerUnit'] > 38000):
    p = setOnDemandPrice(row)
    p = getHourlyCostWithPurchaseType(row['PurchaseOption'], row['LeaseContractLength'], p)
    p = setOfferingClassPrice(row['OfferingClass'], p)
    p = p*setLocationPrice(row['Location'])/100
    p = p*setOSPrice(row['Operating_System'])/100
    return p*730 + 10*setStoragePriceWithLocation(row['Location'])/100
  # else :
  #   return row['PricePerUnit']

def setPricingUpfrontEC2(row):
    p = setOnDemandPrice(row)
    p = getUpfrontCostWithPurchaseType(row['PurchaseOption'], row['LeaseContractLength'], p)
    p = setOfferingClassPrice(row['OfferingClass'], p)
    p = p*setLocationPrice(row['Location'])/100
    p = p*setOSPrice(row['Operating_System'])/100
    return p


df1['PricePerUnit'] = df1.apply (lambda row: setPricingEC2(row), axis=1) 
df1['UpfrontCost'] = df1.apply (lambda row: setPricingUpfrontEC2(row), axis=1) 

df1['PricePerUnit'] = df1['PricePerUnit'].apply(lambda x: round(x, 2))
df1['UpfrontCost'] = df1['UpfrontCost'].apply(lambda x: round(x, 2))

df1.head(5) 

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass
0,10.00,1831.75,m4.xlarge,RHEL,US East (Ohio),1yr,NoUpfront,standard
1,26.08,216.91,m4.large,Linux,US East (N. Virginia),3yr,PartialUpfront,convertible
4,498.35,0.00,m3.xlarge,RHEL,US West (N. California),OnDemand,OnDemand,OnDemand
5,178.75,0.00,m5ad.2xlarge,Windows,US East (N. Virginia),3yr,AllUpfront,convertible
8,145.75,0.00,a1,Linux,US West (Oregon),1yr,AllUpfront,standard


In [8]:
df1_a1 = df1.copy()
df1_a1['Instance_Type'] = df1_a1.apply (lambda row: get_a1_ec2(row), axis=1)

df1_a1 = df1_a1.dropna(subset=['Instance_Type'], axis=0)
df1_a1

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass
8,145.75,0.00,a1.medium,Linux,US West (Oregon),1yr,AllUpfront,standard
1903,126.10,1407.56,a1.4xlarge,RHEL,EU (Ireland),3yr,PartialUpfront,standard
2152,8.00,233.55,a1.medium,RHEL,US West (Oregon),1yr,NoUpfront,standard
2459,79.85,0.00,a1.large,SUSE,US West (Oregon),OnDemand,OnDemand,OnDemand
2650,365.39,0.00,a1.4xlarge,RHEL,US East (N. Virginia),1yr,AllUpfront,convertible
...,...,...,...,...,...,...,...,...
804333,61.61,0.00,a1.xlarge,Linux,US East (N. Virginia),1yr,AllUpfront,convertible
804481,79.17,854.03,a1.2xlarge,SUSE,US East (N. Virginia),3yr,PartialUpfront,convertible
804557,30.34,268.04,a1.large,RHEL,US West (Oregon),1yr,PartialUpfront,convertible
805523,8.00,280.25,a1.large,Linux,US West (Oregon),1yr,NoUpfront,standard


In [9]:
# df1_t4g = df1.copy()
# df1_t4g['Instance_Type'] = df1_t4g.apply (lambda row: get_t4g_ec2(row), axis=1)

# df1_t4g = df1_t4g.dropna(subset=['Instance_Type'], axis=0)
# df1_t4g

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass


In [10]:
df1_t3 = df1.copy()
df1_t3['Instance_Type'] = df1_t3.apply (lambda row: get_t3_ec2(row), axis=1)

df1_t3 = df1_t3.dropna(subset=['Instance_Type'], axis=0)
df1_t3

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass
69,42.68,368.15,t3.large,RHEL,Asia Pacific (Singapore),3yr,PartialUpfront,standard
148,31.60,0.00,t3.medium,Windows,EU (Frankfurt),3yr,AllUpfront,convertible
183,40.55,343.77,t3.large,RHEL,EU (Frankfurt),3yr,PartialUpfront,convertible
296,29.11,223.91,t3.medium,SUSE,EU (Stockholm),3yr,PartialUpfront,convertible
345,295.51,0.00,t3.2xlarge,Windows,Asia Pacific (Sydney),1yr,AllUpfront,convertible
...,...,...,...,...,...,...,...,...
805507,89.12,910.99,t3.xlarge,Windows,Asia Pacific (Hong Kong),1yr,PartialUpfront,standard
805566,12.16,25.86,t3.micro,Windows,US East (Ohio),3yr,PartialUpfront,convertible
805619,373.70,0.00,t3.2xlarge,Windows,AWS GovCloud (US-East),OnDemand,OnDemand,OnDemand
805627,15.26,87.08,t3.small,SUSE,US East (N. Virginia),3yr,PartialUpfront,convertible


In [11]:
df1_t3a = df1.copy()
df1_t3a['Instance_Type'] = df1_t3a.apply (lambda row: get_t3a_ec2(row), axis=1)

df1_t3a = df1_t3a.dropna(subset=['Instance_Type'], axis=0)
df1_t3a

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass
20,11.90,775.95,t3a.large,SUSE,Asia Pacific (Sydney),3yr,NoUpfront,standard
43,8.00,652.47,t3a.xlarge,Linux,US West (Oregon),3yr,NoUpfront,convertible
237,18.93,83.19,t3a.small,RHEL,Asia Pacific (Singapore),3yr,PartialUpfront,standard
331,172.28,0.00,t3a.2xlarge,Windows,Asia Pacific (Tokyo),3yr,AllUpfront,convertible
342,30.72,0.00,t3a.small,RHEL,EU (Frankfurt),1yr,AllUpfront,convertible
...,...,...,...,...,...,...,...,...
805123,11.74,29.55,t3a.micro,Windows,EU (London),3yr,PartialUpfront,convertible
805227,115.47,0.00,t3a.xlarge,Windows,EU (Frankfurt),1yr,AllUpfront,convertible
805235,24.96,0.00,t3a.small,SUSE,EU (Frankfurt),3yr,AllUpfront,standard
805438,227.58,0.00,t3a.2xlarge,Linux,US West (Oregon),OnDemand,OnDemand,OnDemand


In [12]:
# df1_m6g = df1.copy()
# df1_m6g['Instance_Type'] = df1_m6g.apply (lambda row: get_m6g_ec2(row), axis=1)

# df1_m6g = df1_m6g.dropna(subset=['Instance_Type'], axis=0)
# df1_m6g

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass


In [13]:
df1_m5 = df1.copy()
df1_m5['Instance_Type'] = df1_m5.apply (lambda row: get_m5_ec2(row), axis=1)

df1_m5 = df1_m5.dropna(subset=['Instance_Type'], axis=0)
df1_m5

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass
35,7649.87,0.00,m5d.metal,SUSE,US East (N. Virginia),OnDemand,OnDemand,OnDemand
46,2723.45,0.00,m5.metal,Linux,EU (Stockholm),1yr,AllUpfront,standard
50,104.84,0.00,m5.xlarge,Linux,Asia Pacific (Mumbai),OnDemand,OnDemand,OnDemand
56,640.75,0.00,m5d.4xlarge,SUSE,US East (Ohio),3yr,AllUpfront,convertible
64,2671.70,31934.94,m5d.24xlarge,RHEL,EU (Stockholm),1yr,PartialUpfront,standard
...,...,...,...,...,...,...,...,...
805673,160.83,0.00,m5.xlarge,Windows,AWS GovCloud (US),1yr,AllUpfront,convertible
805686,167.15,1909.77,m5.4xlarge,Windows,US East (N. Virginia),3yr,PartialUpfront,convertible
805689,495.26,5806.28,m5.metal,Linux,Asia Pacific (Mumbai),3yr,PartialUpfront,standard
805695,1346.78,0.00,m5d.8xlarge,Linux,Asia Pacific (Singapore),1yr,AllUpfront,convertible


In [14]:
# df1_m5a = df1.copy()
# df1_m5a['Instance_Type'] = df1_m5a.apply (lambda row: get_m5a_ec2(row), axis=1)

# df1_m5a = df1_m5a.dropna(subset=['Instance_Type'], axis=0)
# df1_m5a

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass


In [53]:
# df1_m5n = df1.copy()
# df1_m5n['Instance_Type'] = df1_m5n.apply (lambda row: get_m5n_ec2(row), axis=1)

# df1_m5n = df1_m5n.dropna(subset=['Instance_Type'], axis=0)
# df1_m5n

NameError: ignored

In [15]:
df1_m4 = df1.copy()
df1_m4['Instance_Type'] = df1_m4.apply (lambda row: get_m4_ec2(row), axis=1)

df1_m4 = df1_m4.dropna(subset=['Instance_Type'], axis=0)
df1_m4

,PricePerUnit,UpfrontCost,Instance_Type,Operating_System,Location,LeaseContractLength,PurchaseOption,OfferingClass
0,10.00,1831.75,m4.xlarge,RHEL,US East (Ohio),1yr,NoUpfront,standard
1,26.08,216.91,m4.large,Linux,US East (N. Virginia),3yr,PartialUpfront,convertible
27,1473.37,0.00,m4.16xlarge,Linux,US West (Oregon),1yr,AllUpfront,standard
57,1208.96,14375.47,m4.16xlarge,RHEL,Canada (Central),3yr,PartialUpfront,convertible
197,46.40,0.00,m4.large,Windows,Asia Pacific (Mumbai),1yr,AllUpfront,standard
...,...,...,...,...,...,...,...,...
805630,679.35,0.00,m4.4xlarge,Linux,EU (Frankfurt),OnDemand,OnDemand,OnDemand
805643,35.48,288.90,m4.xlarge,Windows,Asia Pacific (Mumbai),3yr,PartialUpfront,standard
805659,1213.02,0.00,m4.10xlarge,Windows,US West (Oregon),1yr,AllUpfront,convertible
805681,10.00,630.68,m4.large,Linux,US East (Ohio),1yr,NoUpfront,convertible


In [16]:
# df1_a1.value_counts().to_dict()

In [17]:
# !pip install category_encoder
!pip install --upgrade category_encoders

     |████████████████████████████████| 81kB 3.2MB/s 


In [18]:
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential 
from sklearn.linear_model import LogisticRegression
from keras.layers import Dense
from keras import regularizers
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from statistics import mean, stdev 
from sklearn import linear_model
import statsmodels.api as sm
import tkinter as tk 
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.model_selection import cross_val_score
## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost
classifier=xgboost.XGBClassifier()
import keras
import tensorflow as tf
import sklearn.metrics as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [19]:
encoder = ce.OneHotEncoder(cols=['Instance_Type', 'Operating_System', 'Location', 'LeaseContractLength', 'PurchaseOption', 'OfferingClass'], use_cat_names=True)

In [ ]:
# df1_a1_encoded = encoder.fit_transform(df1_a1)
# df1_a1_X = df1_a1_encoded.drop(columns=['PricePerUnit', 'UpfrontCost']).values
# df1_a1_y = df1_a1_encoded.PricePerUnit.values
# df1_a1_training_X, df1_a1_testing_X, df1_a1_training_y, df1_a1_testing_y = train_test_split(df1_a1_X,df1_a1_y , test_size=0.20,random_state=1)

# def baseline_model():
#   print('Multi Layer Perceptron Deep Learning Model')
#   model = keras.models.Sequential()

#   model.add(keras.layers.Dense(22, input_dim=22, activation='relu'))
#   model.add(keras.layers.Dense(22, activation='relu'))
#   model.add(keras.layers.Dense(1))

#   model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
#   return model


# df1_a1_model = baseline_model()
# # fit model
# df1_a1_history = df1_a1_model.fit(df1_a1_training_X, df1_a1_training_y, epochs=100, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],validation_data=(df1_a1_testing_X, df1_a1_testing_y),batch_size=1,verbose=True)

# loss_and_metrics = df1_a1_model.evaluate(df1_a1_testing_X,df1_a1_testing_y)
# print(loss_and_metrics)

In [ ]:
# df1_a1_encoded.shape

In [ ]:
# df1_a1_testing_pred_y = df1_a1_model.predict(df1_a1_testing_X)
# print("R2 score =", round(sm.r2_score(df1_a1_testing_y, df1_a1_testing_pred_y), 2))

# plt.plot(df1_a1_testing_y, color = 'red', label = 'Real data')
# plt.plot(df1_a1_testing_pred_y, color = 'blue', label = 'Predicted data')
# plt.title('Prediction')
# plt.legend()
# plt.show()

# test_data = np.array([1,	0,	0,	0,	0,	1,	0,	0,	1,	0,	0,	0,	1,	0,	0,	1,	0,	0,	0,	1,	0,	0])
# print(df1_a1_model.predict(test_data.reshape(1,22), batch_size=1))


# # plot loss during training
# plt.title('Mean Squared Error')
# plt.plot(df1_a1_history.history['loss'], label='train')
# # plt.plot(history.history['val_loss'], label='test')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend()
# plt.show()

In [ ]:
# df1_t3_encoded = encoder.fit_transform(df1_t3)
# print(df1_t3_encoded.shape)
# df1_t3_X = df1_t3_encoded.drop(columns=['PricePerUnit', 'UpfrontCost']).values
# df1_t3_y = df1_t3_encoded.PricePerUnit.values
# df1_t3_training_X, df1_t3_testing_X, df1_t3_training_y, df1_t3_testing_y = train_test_split(df1_t3_X,df1_t3_y , test_size=0.20,random_state=1)

# def baseline_model():
#   print('Multi Layer Perceptron Deep Learning Model')
#   model = keras.models.Sequential()

#   model.add(keras.layers.Dense(41, input_dim=41, activation='relu'))
#   model.add(keras.layers.Dense(41, activation='relu'))
#   model.add(keras.layers.Dense(1))

#   model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae','mse','mape'])
#   return model


# df1_t3_model = baseline_model()
# # fit model
# df1_t3_history = df1_t3_model.fit(df1_t3_training_X, df1_t3_training_y, epochs=20, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],validation_data=(df1_t3_testing_X, df1_t3_testing_y),batch_size=1,verbose=True)

# loss_and_metrics = df1_t3_model.evaluate(df1_t3_testing_X,df1_t3_testing_y)
# print(loss_and_metrics)

In [ ]:
# df1_t3_encoded

In [ ]:
# df1_t3_encoded.iloc[15876].values[2:]

In [ ]:
# df1_t3_testing_pred_y = df1_t3_model.predict(df1_t3_testing_X)
# print("R2 score =", round(sm.r2_score(df1_t3_testing_y, df1_t3_testing_pred_y), 2))

# plt.plot(df1_t3_testing_y, color = 'red', label = 'Real data')
# plt.plot(df1_t3_testing_pred_y, color = 'blue', label = 'Predicted data')
# plt.title('Prediction')
# plt.legend()
# plt.show()

# test_data = df1_t3_encoded.iloc[15876].values[2:]
# print(df1_t3_model.predict(test_data.reshape(1,41), batch_size=1))


# # plot loss during training
# plt.title('Mean Squared Error')
# plt.plot(df1_t3_history.history['loss'], label='train')
# # plt.plot(history.history['val_loss'], label='test')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend()
# plt.show()

In [ ]:
# df1_t3a_encoded = encoder.fit_transform(df1_t3a)
# df1_t3a_X = df1_t3a_encoded.drop(columns=['PricePerUnit', 'UpfrontCost']).values
# df1_t3a_y = df1_t3a_encoded.PricePerUnit.values
# df1_t3a_training_X, df1_t3a_testing_X, df1_t3a_training_y, df1_t3a_testing_y = train_test_split(df1_t3a_X,df1_t3a_y , test_size=0.20,random_state=1)

# def baseline_model():
#   print('Multi Layer Perceptron Deep Learning Model')
#   model = keras.models.Sequential()

#   model.add(keras.layers.Dense(36, input_dim=36, activation='relu'))
#   model.add(keras.layers.Dense(36, activation='relu'))
#   model.add(keras.layers.Dense(1))

#   model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
#   return model


# df1_t3a_model = baseline_model()
# # fit model
# df1_t3a_history = df1_t3a_model.fit(df1_t3a_training_X, df1_t3a_training_y, epochs=100, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],validation_data=(df1_t3a_testing_X, df1_t3a_testing_y),batch_size=1,verbose=True)

# loss_and_metrics = df1_t3a_model.evaluate(df1_t3a_testing_X,df1_t3a_testing_y)
# print(loss_and_metrics)

In [38]:
# df1_t3a_encoded.shape[1]

38

In [ ]:
# df1_t3a_encoded.iloc[1].values[2:]

In [ ]:
# df1_t3a_testing_pred_y = df1_t3a_model.predict(df1_t3a_testing_X)
# print("R2 score =", round(sm.r2_score(df1_t3a_testing_y, df1_t3a_testing_pred_y), 2))

# plt.plot(df1_t3a_testing_y, color = 'red', label = 'Real data')
# plt.plot(df1_t3a_testing_pred_y, color = 'blue', label = 'Predicted data')
# plt.title('Prediction')
# plt.legend()
# plt.show()

# df1_t3a_test_data = df1_t3a_encoded.iloc[1].values[2:]
# print(df1_t3a_model.predict(df1_t3a_test_data.reshape(1,36), batch_size=1))


# # plot loss during training
# plt.title('Mean Squared Error')
# plt.plot(df1_t3a_history.history['loss'], label='train')
# # plt.plot(history.history['val_loss'], label='test')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend()
# plt.show()

In [30]:
# df1_m6g_encoded = encoder.fit_transform(df1_m6g)
# df1_m6g_X = df1_m6g_encoded.drop(columns=['PricePerUnit', 'UpfrontCost']).values
# df1_m6g_y = df1_m6g_encoded.PricePerUnit.values
# df1_m6g_training_X, df1_m6g_testing_X, df1_m6g_training_y, df1_m6g_testing_y = train_test_split(df1_m6g_X,df1_m6g_y , test_size=0.20,random_state=1)

# def baseline_model():
#   print('Multi Layer Perceptron Deep Learning Model')
#   model = keras.models.Sequential()

#   model.add(keras.layers.Dense(36, input_dim=36, activation='relu'))
#   model.add(keras.layers.Dense(36, activation='relu'))
#   model.add(keras.layers.Dense(1))

#   model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
#   return model


# df1_m6g_model = baseline_model()
# # fit model
# df1_m6g_history = df1_m6g_model.fit(df1_m6g_training_X, df1_m6g_training_y, epochs=100, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],validation_data=(df1_m6g_testing_X, df1_m6g_testing_y),batch_size=1,verbose=True)

# loss_and_metrics = df1_m6g_model.evaluate(df1_m6g_testing_X,df1_m6g_testing_y)
# print(loss_and_metrics)

In [ ]:
# df1_m5_encoded = encoder.fit_transform(df1_m5)
# df1_m5_X = df1_m5_encoded.drop(columns=['PricePerUnit', 'UpfrontCost']).values
# df1_m5_y = df1_m5_encoded.PricePerUnit.values
# df1_m5_training_X, df1_m5_testing_X, df1_m5_training_y, df1_m5_testing_y = train_test_split(df1_m5_X,df1_m5_y , test_size=0.20,random_state=1)

# def baseline_model():
#   print('Multi Layer Perceptron Deep Learning Model')
#   model = keras.models.Sequential()

#   model.add(keras.layers.Dense(20, input_dim=52, activation='relu'))
#   model.add(keras.layers.Dense(20, activation='relu'))
#   model.add(keras.layers.Dense(1))

#   model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
#   return model


# df1_m5_model = baseline_model()
# # fit model
# df1_m5_history = df1_m5_model.fit(df1_m5_training_X, df1_m5_training_y, epochs=100, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],validation_data=(df1_m5_testing_X, df1_m5_testing_y),batch_size=1,verbose=True)

# loss_and_metrics = df1_m5_model.evaluate(df1_m5_testing_X,df1_m5_testing_y)
# print(loss_and_metrics)

In [ ]:
# df1_m5_testing_pred_y = df1_m5_model.predict(df1_m5_testing_X)
# print("R2 score =", round(sm.r2_score(df1_m5_testing_y, df1_m5_testing_pred_y), 2))

# plt.plot(df1_m5_testing_y, color = 'red', label = 'Real data')
# plt.plot(df1_m5_testing_pred_y, color = 'blue', label = 'Predicted data')
# plt.title('Prediction')
# plt.legend()
# plt.show()

# df1_m5_test_data = df1_m5_encoded.iloc[1].values[2:]
# print(df1_m5_model.predict(df1_m5_test_data.reshape(1,52), batch_size=1))


# # plot loss during training
# plt.title('Mean Squared Error')
# plt.plot(df1_m5_history.history['loss'], label='train')
# # plt.plot(history.history['val_loss'], label='test')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend()
# plt.show()

In [ ]:
# df1_m5a_encoded = encoder.fit_transform(df1_m5a)
# df1_m5a_X = df1_m5a_encoded.drop(columns=['PricePerUnit', 'UpfrontCost']).values
# df1_m5a_y = df1_m5a_encoded.PricePerUnit.values
# df1_m5a_training_X, df1_m5a_testing_X, df1_m5a_training_y, df1_m5a_testing_y = train_test_split(df1_m5a_X,df1_m5a_y , test_size=0.20,random_state=1)

# def baseline_model():
#   print('Multi Layer Perceptron Deep Learning Model')
#   model = keras.models.Sequential()

#   model.add(keras.layers.Dense(20, input_dim=52, activation='relu'))
#   model.add(keras.layers.Dense(20, activation='relu'))
#   model.add(keras.layers.Dense(1))

#   model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
#   return model


# df1_m5a_model = baseline_model()
# # fit model
# df1_m5a_history = df1_m5a_model.fit(df1_m5a_training_X, df1_m5a_training_y, epochs=100, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],validation_data=(df1_m5a_testing_X, df1_m5a_testing_y),batch_size=1,verbose=True)

# loss_and_metrics = df1_m5a_model.evaluate(df1_m5a_testing_X,df1_m5a_testing_y)
# print(loss_and_metrics)

In [ ]:
# df1_m5a_testing_pred_y = df1_m5a_model.predict(df1_m5a_testing_X)
# print("R2 score =", round(sm.r2_score(df1_m5a_testing_y, df1_m5a_testing_pred_y), 2))

# plt.plot(df1_m5a_testing_y, color = 'red', label = 'Real data')
# plt.plot(df1_m5a_testing_pred_y, color = 'blue', label = 'Predicted data')
# plt.title('Prediction')
# plt.legend()
# plt.show()

# df1_m5a_test_data = df1_m5a_encoded.iloc[1].values[2:]
# print(df1_m5a_model.predict(df1_m5a_test_data.reshape(1,52), batch_size=1))


# # plot loss during training
# plt.title('Mean Squared Error')
# plt.plot(df1_m5a_history.history['loss'], label='train')
# # plt.plot(history.history['val_loss'], label='test')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend()
# plt.show()

In [ ]:
# df1_m4_encoded = encoder.fit_transform(df1_m4)
# df1_m4_X = df1_m4_encoded.drop(columns=['PricePerUnit', 'UpfrontCost']).values
# df1_m4_y = df1_m4_encoded.PricePerUnit.values
# df1_m4_training_X, df1_m4_testing_X, df1_m4_training_y, df1_m4_testing_y = train_test_split(df1_m4_X,df1_m4_y , test_size=0.20,random_state=1)

# def baseline_model():
#   print('Multi Layer Perceptron Deep Learning Model')
#   model = keras.models.Sequential()

#   model.add(keras.layers.Dense(20, input_dim=52, activation='relu'))
#   model.add(keras.layers.Dense(20, activation='relu'))
#   model.add(keras.layers.Dense(1))

#   model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
#   return model


# df1_m4_model = baseline_model()
# # fit model
# df1_m4_history = df1_m4_model.fit(df1_m4_training_X, df1_m4_training_y, epochs=100, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],validation_data=(df1_m4_testing_X, df1_m4_testing_y),batch_size=1,verbose=True)

# loss_and_metrics = df1_m4_model.evaluate(df1_m4_testing_X,df1_m4_testing_y)
# print(loss_and_metrics)

In [ ]:
# df1_m4_testing_pred_y = df1_m4_model.predict(df1_m4_testing_X)
# print("R2 score =", round(sm.r2_score(df1_m4_testing_y, df1_m4_testing_pred_y), 2))

# plt.plot(df1_m4_testing_y, color = 'red', label = 'Real data')
# plt.plot(df1_m4_testing_pred_y, color = 'blue', label = 'Predicted data')
# plt.title('Prediction')
# plt.legend()
# plt.show()

# df1_m4_test_data = df1_m4_encoded.iloc[1].values[2:]
# print(df1_m4_model.predict(df1_m4_test_data.reshape(1,52), batch_size=1))


# # plot loss during training
# plt.title('Mean Squared Error')
# plt.plot(df1_m4_history.history['loss'], label='train')
# # plt.plot(history.history['val_loss'], label='test')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend()
# plt.show()

In [55]:
df1_array = [df1_a1, df1_t3, df1_t3a, df1_m5, df1_m4]
df1_title_array = ['A1','T3', 'T3a', 'M5', 'M4']
df1_models_array = []
df1_history_array = []
df1_loss_and_metrics_array = []

def baseline_model(input_dim_number):
  print('Multi Layer Perceptron Deep Learning Model for ' + df1_title_array[i]  + ' Instance type price calculation:')
  model = keras.models.Sequential()
  model.add(keras.layers.Dense(input_dim_number, input_dim=input_dim_number, activation='relu'))
  model.add(keras.layers.Dense(input_dim_number, activation='relu'))
  model.add(keras.layers.Dense(1))
  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
  return model

i = 0
for df1_model in df1_array:
  df1_encoded = encoder.fit_transform(df1_model)
  df1_X = df1_encoded.drop(columns=['PricePerUnit', 'UpfrontCost']).values
  df1_y = df1_encoded.PricePerUnit.values
  df1_training_X, df1_testing_X, df1_training_y, df1_testing_y = train_test_split(df1_X,df1_y , test_size=0.20,random_state=1)
  df1_models_array.append(baseline_model(df1_encoded.shape[1] - 2))
  df1_history_array.append(df1_models_array[i].fit(df1_training_X, df1_training_y, epochs=30, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],validation_data=(df1_testing_X, df1_testing_y),batch_size=1,verbose=False))
  df1_loss_and_metrics_array.append(df1_models_array[i].evaluate(df1_testing_X,df1_testing_y))
  print(df1_loss_and_metrics_array[i])
  df1_testing_pred_y = df1_models_array[i].predict(df1_testing_X)
  print(df1_title_array[i] +" - R2 score =", round(sm.r2_score(df1_testing_y, df1_testing_pred_y), 2))

  # plt.plot(df1_testing_y, color = 'red', label = 'Real data')
  # plt.plot(df1_testing_pred_y, color = 'blue', label = 'Predicted data')
  # plt.title(df1_title_array[i] + ' Prediction')
  # plt.legend()
  # plt.show()
 
  df1_test_data = df1_encoded.iloc[1].values[2:]
  print(df1_title_array[i] + ' Prediction Results: ')
  print('X: ' , df1_test_data)
  print('Y: $' , df1_models_array[i].predict(df1_test_data.reshape(1,df1_encoded.shape[1] - 2), batch_size=1)[0][0])

  # plt.title(df1_title_array[i] + ' Mean Squared Error')
  # plt.plot(df1_history_array[i].history['loss'], label='train')
  # plt.ylabel('loss')
  # plt.xlabel('epoch')
  # plt.legend()
  # plt.show()
  i = i + 1




/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Multi Layer Perceptron Deep Learning Model for A1 Instance type price calculation:
14/14 [==============================] - 0s 2ms/step - loss: 150.2800 - mean_squared_error: 150.2800
[150.28004455566406, 150.28004455566406]
A1 - R2 score = 0.99
A1 Prediction Results: 
X:  [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0.]
Y: $ 123.88976


/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Multi Layer Perceptron Deep Learning Model for T3 Instance type price calculation:
100/100 [==============================] - 0s 1ms/step - loss: 5.7893 - mean_squared_error: 5.7893
[5.789290428161621, 5.789290428161621]
T3 - R2 score = 1.0
T3 Prediction Results: 
X:  [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
Y: $ 30.77834
Multi Layer Perceptron Deep Learning Model for T3a Instance type price calculation:


/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


76/76 [==============================] - 0s 1ms/step - loss: 3.5836 - mean_squared_error: 3.5836
[3.583615779876709, 3.583615779876709]
T3a - R2 score = 1.0
T3a Prediction Results: 
X:  [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
Y: $ 8.097546


/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Multi Layer Perceptron Deep Learning Model for M5 Instance type price calculation:
723/723 [==============================] - 1s 1ms/step - loss: 204.2993 - mean_squared_error: 204.2993
[204.29933166503906, 204.29933166503906]
M5 - R2 score = 1.0
M5 Prediction Results: 
X:  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 1. 0.]
Y: $ 2690.9863


/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Multi Layer Perceptron Deep Learning Model for M4 Instance type price calculation:
195/195 [==============================] - 0s 1ms/step - loss: 52.7093 - mean_squared_error: 52.7093
[52.70934295654297, 52.70934295654297]
M4 - R2 score = 1.0
M4 Prediction Results: 
X:  [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0.]
Y: $ 23.759607
